PILAR ARIAS, YEDRA MARTÍN, PATRICIA MEDIAVILLA, AMAYA SARASA

<GRUPO 9>

#Práctica 1

##Introducción

Esta es la primera práctica del segundo laboratorio. Dada una base de datos en formato .csv, se nos pide contestar a una serie de cuestiones. Para ello, lo priemro que debemos hacer es cargar y leer la base de datos:

In [2]:
#Importar librerías
#!apt install mlxtend
import numpy as np #numpy
import pandas as pd #pandas
import sklearn as sk #scikit-learn
import matplotlib.pyplot as plt #gráfica Matplotlib
#from mlxtend.frequent_patterns import apriori
#from apyori import apriori #apriori
from mlxtend.preprocessing import TransactionEncoder
#from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori, association_rules
#from mlxtend.frequent_patterns import frequent_patterns
#from mlxtend import frequent_patterns
#from mlxtend.frequent_patterns import frequent_patterns

#Importamos la base de datos
df = pd.read_csv('BlackFriday.csv',sep=',')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102072 entries, 0 to 102071
Data columns (total 11 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   User_ID                     102072 non-null  int64  
 1   Product_ID                  102072 non-null  object 
 2   Gender                      102071 non-null  object 
 3   Age                         102071 non-null  object 
 4   Occupation                  102071 non-null  float64
 5   City_Category               102071 non-null  object 
 6   Stay_In_Current_City_Years  102071 non-null  object 
 7   Marital_Status              102071 non-null  float64
 8   Product_Category_1          102071 non-null  float64
 9   Product_Category_2          70544 non-null   float64
 10  Product_Category_3          31065 non-null   float64
dtypes: float64(5), int64(1), object(5)
memory usage: 8.6+ MB


,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000004,P00128942,M,46-50,7.0,B,2,1.0,1.0,11.0,NaN
1,1000009,P00113442,M,26-35,17.0,C,0,0.0,3.0,5.0,NaN
2,1000010,P00288442,F,36-45,1.0,B,4+,1.0,5.0,14.0,NaN
3,1000010,P00145342,F,36-45,1.0,B,4+,1.0,4.0,9.0,NaN
4,1000011,P00053842,F,26-35,1.0,C,1,0.0,4.0,5.0,12.0


----------------

----------------

Subimos la base de datos, df.info nos aporta la infomación que contiene la base de datos, vemos el tipo, la cantidad de las variables con las que trabajamos... y .head vamos a ver que forma tiene la base de datos y las primeras filas de esta.

Según se ha estudiado en clase, encontramos dos métodos para afrontarlo. Nos decantaremos por CRISP-DM. Las siglas para: CRoss-Industry Standard Process for Data Mining. 

Es un modelo de proceso de minería de datos que se utiliza en la industria. Es un marco de trabajo que define un proceso estandarizado para llevar a cabo proyectos de minería de datos, desde la definición del problema hasta la implementación y evaluación del resultado. Los seis pasos que componen Crisp-DM son los siguientes:

- Comprensión de los negocios y definición del problema: comprender la naturaleza del problema y los objetivos de negocio.

- Comprensión de los datos: recopilación inicial, descripción y exploración  de datos.

- Preparación de los datos: Selección, limpieza, construcción, integración, formateo de los datos.

- Modelado: aplicar las técnicas de minería de datos a los dataset.

- Evaluación: determinar si los resultados son útiles a las necesidades del negocio.

- Despliegue: explotar utilidad de los modelos, integrándolos en las tareas de toma de decisiones de la organización -> Call to Action. 

##Comprensión de los negocios y definición del problema

La librería se usará de la siguiente manera:
1) Empezaremos obteniendo los itemsets frecuentes para k=1. En este punto necesitaremos obtener el
soporte de los itemset. Por lo tanto, se tendrá que usar un método que dado un itemset devuelva su
soporte.

2) Para k≥2.
a. Mostrar los itemset frecuentes candidatos y su soporte.

3) Repetir el proceso 2 hasta que no se generen nuevos itemsets frecuentes.

4) Mostrar todas las posibles reglas con la confianza de cada una de ellas.

5) Listar todas las reglas que sean de alta confianza.

6) Usar los siguientes métodos. Dado un antecesor, devolver todas las reglas que contengan a dicho
antecesor. Dado un umbral mínimo devolver todas las reglas que cumplan con dicha confianza.

7) Utiliza las representaciones gráficas que consideres adecuadas para representar las reglas obtenidas
y obtener conclusiones a partir de los datos.

A partir de lo anterior de pide:
- Prueba al menos tres configuraciones de soporte y frecuencia para cada género, edad y tipo de producto.
- ¿Qué diferencias hay entre usar soporte y frecuencia? Respalda la respuesta con datos 
- ¿Qué tipo reglas desaparecen según la configuración y categorías (género, edad y tipo de producto)
consideradas? ¿Por qué? 
- Para una de las configuraciones, interpreta algunas de las reglas que te hayan resultado interesantes
obtenidas usando la clase association_rules y un par de configuraciones cambiando la variable metric y
min_threshold. Justifica los resultados. 
- Dadas las mejores configuraciones. ¿Existen reglas o patrones que se repitan? ¿Podemos generalizar de
alguna manera como se comportan los clientes? Expón las conclusiones respaldadas con datos 

## Comprensión de los datos

La base de datos que utilizaremos consta de 233599 observaciones. Teniendo en ceunta que el trabajo que vamos a realizar, utilizanod el algoritmo apriori, queremos sacar regalas y con todos estos datos, lo que nos hace pensar es que vamos a encontrar muchísimas reglas a partir de estos datos.

La clave está en cuales son las que más nos interesan, basandonos en su confianza, y una vez hallado esto debemos tomar decisiones para que usando estas reglas vendamos el mayor número de producto, para aumentar nuestras gananzas.

Existen 11 tipos de atributos distintos: 

- User_ID: número para identificar al usuario.
- Product_ID: número para identificar el producto.
- Gender: Género del comprador.	
- Age: Edad del comprador.
- Occupation: A lo que se dedica el comprador, ya pasasdo a categórica.
- City_Category: Categoría de la ciudad donde reside el user.
- Stay_In_Current_City_Years: El tiempo que lleva en la ciudad.
- Marital_Status: Estado civil.
- Product_Category_1: La categoría principal del producto.
- Product_Category_2: Siguente categoría.
- Product_Category_3: Última categoría del producto.



##Preparación de los datos

In [3]:
usernan = df['User_ID'].isna().sum()
print(f"La cantidad de NaN que existen en la variable User_ID es:{usernan}")
productnan = df['Product_ID'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Product_ID es:{productnan}")
gendernan = df['Gender'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Gender es:{gendernan}")
agenan = df['Age'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Age es:{agenan}")
ocupationan = df['Occupation'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Occupation es:{ocupationan}")
City_Categorynan = df['City_Category'].isna().sum()
print(f"La cantidad de NaN que existen en la variable City_Category es:{City_Categorynan}")
Stay_In_Current_City_Yearsnan = df['Stay_In_Current_City_Years'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Stay_In_Current_City_Years es:{Stay_In_Current_City_Yearsnan}")
Marital_Statusnan = df['Marital_Status'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Marital_Status es:{Marital_Statusnan}")
Product_Category_1nan = df['Product_Category_1'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Product_Category_1 es:{Product_Category_1nan}")
Product_Category_2nan = df['Product_Category_2'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Product_Category_2 es:{Product_Category_2nan}")
Product_Category_3nan = df['Product_Category_3'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Product_Category_3 es:{Product_Category_3nan}")

La cantidad de NaN que existen en la variable User_ID es:0
La cantidad de NaN que existen en la variable Product_ID es:0
La cantidad de NaN que existen en la variable Gender es:1
La cantidad de NaN que existen en la variable Age es:1
La cantidad de NaN que existen en la variable Occupation es:1
La cantidad de NaN que existen en la variable City_Category es:1
La cantidad de NaN que existen en la variable Stay_In_Current_City_Years es:1
La cantidad de NaN que existen en la variable Marital_Status es:1
La cantidad de NaN que existen en la variable Product_Category_1 es:1
La cantidad de NaN que existen en la variable Product_Category_2 es:31528
La cantidad de NaN que existen en la variable Product_Category_3 es:71007


In [4]:
# Rellenamos los valores na con 0, ya que no tiene categoría
df = df.fillna(0)

# Quitamos los duplicados
df = df.drop_duplicates()

##Modelado

In [5]:
df_User_Product = (df
        .groupby(['User_ID', 'Product_ID'])['Product_Category_1']
        .sum()
        .apply(lambda x:x>0)
        .unstack()
        .fillna(False)
        .reset_index()
        .set_index('User_ID')
          )

In [6]:
df_Gender = (df
            .groupby(['User_ID','Gender'])['Product_Category_1']
            .sum()
            .apply(lambda x: x>0)
            .unstack()
            .fillna(False)
            .reset_index()
            .set_index('User_ID')
)

In [7]:
df_Age = (df
        .groupby(['User_ID','Age'])['Product_Category_1']
        .sum()
        .apply(lambda x: x>0)
        .unstack()
        .fillna(False)
        .reset_index()
        .set_index('User_ID')
)

In [8]:
df_Occupation = (df
                .groupby(['User_ID','Occupation'])['Product_Category_1']
                .sum()
                .apply(lambda x: x>0)
                .unstack()
                .fillna(False)
                .reset_index()
                .set_index('User_ID')
)

In [9]:
df_City = (df
          .groupby(['User_ID','City_Category'])['Product_Category_1']
          .sum()
          .apply(lambda x: x>0)
          .unstack()
          .fillna(False)
          .reset_index()
          .set_index('User_ID')
)

In [10]:
df_Stay = (df
           .groupby(['User_ID','Stay_In_Current_City_Years'])['Product_Category_1']
           .sum()
           .apply(lambda x: x>0)
           .unstack()
           .fillna(False)
           .reset_index()
           .set_index('User_ID')
)

In [11]:
df_Marital = (df
              .groupby(['User_ID','Marital_Status'])['Product_Category_1']
              .sum()
              .apply(lambda x: x>0)
              .unstack()
              .fillna(False)
              .reset_index()
              .set_index('User_ID')
)

In [12]:
df_Product_Category_1 = (df
                         .groupby(['User_ID','Product_Category_1'])['Product_Category_2']
                         .sum()
                         .apply(lambda x: x>0)
                         .unstack()
                         .fillna(False)
                         .reset_index()
                         .set_index('User_ID')
)

In [13]:
df_Product_Category_2 = (df
                         .groupby(['User_ID','Product_Category_2'])['Product_Category_1']
                         .sum()
                         .apply(lambda x: x>0)
                         .unstack()
                         .fillna(False)
                         .reset_index()
                         .set_index('User_ID')
)

In [14]:
df_Product_Category_3 = (df
                         .groupby(['User_ID','Product_Category_3'])['Product_Category_1']
                         .sum()
                         .apply(lambda x: x>0)
                         .unstack()
                         .fillna(False)
                         .reset_index()
                         .set_index('User_ID')
)

In [15]:
df_Product_Category_3.value_counts()

0.0   3.0    4.0    5.0    6.0    8.0    9.0    10.0   11.0   12.0   13.0   14.0   15.0   16.0   17.0   18.0 
True  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False    782
                                                                                          True   False  False    229
                                                                                          False  True   False    130
                                                                                   True   False  False  False    114
                                                                            True   False  False  False  False    114
                                                                                                                ... 
                    True   False  False  True   False  True   False  False  True   True   True   True   True       1
                                                                     Tr

In [16]:
df_Product_Category_1=df_Product_Category_1.set_axis(['0_PC0','1_PC1','2_PC1','3_PC1','4_PC1','5_PC1','6_PC1','7_PC1','8_PC1','9_PC1','10_PC1','11_PC1','12_PC1','13_PC1','14_PC1','15_PC1','16_PC1','17_PC1','18_PC1'], axis=1)

In [17]:
df_Product_Category_2=df_Product_Category_2.set_axis(['0_PC2','2_PC2','3_PC2','4_PC2','5_PC2','6_PC2','7_PC2','8_PC2','9_PC2','10_PC2','11_PC2','12_PC2','13_PC2','14_PC2','15_PC2','16_PC2','17_PC2','18_PC2'], axis=1)

In [18]:
df_Product_Category_3=df_Product_Category_3.set_axis(['0_PC3','3_PC3','4_PC3','5_PC3','6_PC3','8_PC3','9_PC3','10_PC3','11_PC3','12_PC3','13_PC3','14_PC3','15_PC3','16_PC3','17_PC3','18_PC3'],axis=1)

In [19]:
df_Marital=df_Marital.set_axis(['0_Marital', '1_Marital'], axis=1)

In [20]:
df_Occupation=df_Occupation.set_axis(['0_Occ','1_Occ', '2_Occ', '3_Occ', '4_Occ', '5_Occ', '6_Occ', '7_Occ', '8_Occ', '9_Occ', '10_Occ', '11_Occ', '12_Occ', '13_Occ', '14_Occ', '15_Occ', '16_Occ', '17_Occ', '18_Occ', '19_Occ', '20_Occ'], axis=1)

In [21]:
df_Stay=df_Stay.set_axis(['0_Stay','1_Stay','2_Stay','3_Stay','4_Stay','5_Stay'],axis=1)

In [22]:
data = pd.concat([df_User_Product, df_Occupation, df_City, df_Stay, df_Marital, df_Product_Category_1, df_Product_Category_2, df_Product_Category_3, df_Age, df_Gender], axis = 1)
data

,P0,P00000142,P00000242,P00000342,P00000442,P00000542,P00000642,P00000742,P00000842,P00000942,...,0-17,18-25,26-35,36-45,46-50,51-55,55+,0,F,M
User_ID,,,,,,,,,,,,,,,,,,,,,
1000001,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
1000002,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True
1000003,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
1000004,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True
1000005,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1006036,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False
1006037,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,True,False
1006038,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False


In [23]:
print(data.shape)

(5828, 3402)


In [24]:
#Esto es para k=1 // use_colnames = True, max_len = 3
#frq_items = apriori(df, min_support = 0.05, use_colnames = True, max_len = 3)

frequent_itemsets = apriori(data, min_support=0.05, use_colnames=True, max_len=3)
frequent_itemsets

,support,itemsets
0,0.056452,(P00025442)
1,0.055765,(P00112142)
2,0.050103,(P00184942)
3,0.062286,(P00265242)
4,0.116507,(0_Occ)
...,...,...
12433,0.146706,"(M, 16_PC3, 26-35)"
12434,0.072066,"(36-45, M, 16_PC3)"
12435,0.053535,"(18_PC3, 17_PC3, M)"
12436,0.094029,"(17_PC3, M, 26-35)"


In [25]:
#Para k=1

frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets1=frequent_itemsets[frequent_itemsets['length'] == 1]
frequent_itemsets1

,support,itemsets,length
0,0.056452,(P00025442),1
1,0.055765,(P00112142),1
2,0.050103,(P00184942),1
3,0.062286,(P00265242),1
4,0.116507,(0_Occ),1
...,...,...,...
68,0.089911,(46-50),1
69,0.081503,(51-55),1
70,0.062800,(55+),1
71,0.282086,(F),1


In [26]:
#Para k=2

frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets2=frequent_itemsets[frequent_itemsets['length'] == 2]
frequent_itemsets2

,support,itemsets,length
73,0.056452,"(1_PC1, P00025442)",2
74,0.056452,"(2_PC2, P00025442)",2
75,0.055765,"(0_PC3, P00025442)",2
76,0.056452,"(9_PC3, P00025442)",2
77,0.055765,"(1_PC1, P00112142)",2
...,...,...,...
1345,0.256349,"(M, 26-35)",2
1346,0.056452,"(36-45, F)",2
1347,0.141558,"(36-45, M)",2
1348,0.059540,"(46-50, M)",2


In [27]:
#Para k=3

frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets3=frequent_itemsets[frequent_itemsets['length'] == 3]
frequent_itemsets3

,support,itemsets,length
1350,0.056452,"(2_PC2, 1_PC1, P00025442)",3
1351,0.055765,"(0_PC3, 1_PC1, P00025442)",3
1352,0.056452,"(9_PC3, 1_PC1, P00025442)",3
1353,0.055765,"(2_PC2, 0_PC3, P00025442)",3
1354,0.056452,"(2_PC2, 9_PC3, P00025442)",3
...,...,...,...
12433,0.146706,"(M, 16_PC3, 26-35)",3
12434,0.072066,"(36-45, M, 16_PC3)",3
12435,0.053535,"(18_PC3, 17_PC3, M)",3
12436,0.094029,"(17_PC3, M, 26-35)",3


In [28]:
#Reglas de asociación
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.8)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(P00025442),(1_PC1),0.056452,0.779856,0.056452,1.000000,1.282288,0.012427,inf
1,(P00025442),(2_PC2),0.056452,0.568291,0.056452,1.000000,1.759662,0.024371,inf
2,(P00025442),(0_PC3),0.056452,0.973747,0.055765,0.987842,1.014475,0.000796,2.159274
3,(P00025442),(9_PC3),0.056452,0.255662,0.056452,1.000000,3.911409,0.042019,inf
4,(P00112142),(1_PC1),0.055765,0.779856,0.055765,1.000000,1.282288,0.012276,inf
...,...,...,...,...,...,...,...,...,...
9033,"(15_PC3, 18-25)",(M),0.079444,0.717914,0.063830,0.803456,1.119154,0.006796,1.435231
9034,"(15_PC3, 26-35)",(M),0.154255,0.717914,0.126802,0.822024,1.145019,0.016060,1.584973
9035,"(18_PC3, 17_PC3)",(16_PC3),0.066575,0.491592,0.053363,0.801546,1.630510,0.020635,2.561847
9036,"(18_PC3, 16_PC3)",(M),0.086651,0.717914,0.070693,0.815842,1.136406,0.008486,1.531760


Pero tenemos que quitar aquellas reglas que tengan un 0 en el antecedente o en la consecuencia. 

In [43]:
mask = (rules['antecedents'].apply(lambda x: '0_PC2' in x) | rules['consequents'].apply(lambda x: '0_PC2' in x) | rules['antecedents'].apply(lambda x: '0_PC3' in x) | rules['consequents'].apply(lambda x: '0_PC3' in x)| rules['antecedents'].apply(lambda x: '0_PC1' in x) | rules['consequents'].apply(lambda x: '0_PC1' in x))


rules_with_zero = rules[mask]

In [44]:
rules = rules.drop(rules_with_zero.index)

In [45]:
#vemos reglas de alta confianza
rulesaltaconfianza = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.95)
rulesaltaconfianza

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(P00025442),(1_PC1),0.056452,0.779856,0.056452,1.000000,1.282288,0.012427,inf
1,(P00025442),(2_PC2),0.056452,0.568291,0.056452,1.000000,1.759662,0.024371,inf
2,(P00025442),(0_PC3),0.056452,0.973747,0.055765,0.987842,1.014475,0.000796,2.159274
3,(P00025442),(9_PC3),0.056452,0.255662,0.056452,1.000000,3.911409,0.042019,inf
4,(P00112142),(1_PC1),0.055765,0.779856,0.055765,1.000000,1.282288,0.012276,inf
...,...,...,...,...,...,...,...,...,...
2664,"(M, 26-35)",(0_PC3),0.256349,0.973747,0.250515,0.977242,1.003589,0.000896,1.153569
2665,"(36-45, F)",(0_PC3),0.056452,0.973747,0.054564,0.966565,0.992624,-0.000405,0.785191
2666,"(36-45, M)",(0_PC3),0.141558,0.973747,0.138298,0.976970,1.003309,0.000456,1.139914
2667,"(46-50, M)",(0_PC3),0.059540,0.973747,0.058339,0.979827,1.006244,0.000362,1.301378


In [32]:
for i in range(len(rulesaltaconfianza)):
  if 'M' in rulesaltaconfianza['antecedents'][i]:
    print(rulesaltaconfianza.iloc[i])

antecedents                (M)
consequents            (0_PC3)
antecedent support    0.717914
consequent support    0.973747
support               0.700927
confidence            0.976338
lift                  1.002661
leverage               0.00186
conviction            1.109503
Name: 95, dtype: object
antecedents           (0_Occ, M)
consequents              (0_PC3)
antecedent support      0.078415
consequent support      0.973747
support                 0.076356
confidence              0.973742
lift                    0.999994
leverage                    -0.0
conviction              0.999786
Name: 169, dtype: object
antecedents           (1_Occ, M)
consequents              (0_PC3)
antecedent support      0.053363
consequent support      0.973747
support                 0.052334
confidence              0.980707
lift                    1.007148
leverage                0.000371
conviction              1.360758
Name: 176, dtype: object
antecedents           (4_Occ, M)
consequents         

In [33]:
for i in range(len(rulesaltaconfianza)):
  if rulesaltaconfianza['confidence'][i] > 0.98:
    print(rulesaltaconfianza.iloc[i])

Se han truncado las últimas 5000 líneas del flujo de salida.
antecedents           (11_PC1, F)
consequents               (0_PC3)
antecedent support       0.052505
consequent support       0.973747
support                  0.052505
confidence                    1.0
lift                      1.02696
leverage                 0.001378
conviction                    inf
Name: 1874, dtype: object
antecedents           (M, 11_PC1)
consequents               (0_PC3)
antecedent support       0.177076
consequent support       0.973747
support                  0.176905
confidence               0.999031
lift                     1.025965
leverage                 0.004477
conviction              27.092656
Name: 1875, dtype: object
antecedents           (0_PC2, 12_PC1)
consequents                  (14_PC2)
antecedent support           0.054907
consequent support           0.584935
support                      0.054049
confidence                   0.984375
lift                          1.68288
leverage 

NO BORRAR: 

In [34]:
#frequent_itemsets = apriori(data, min_support=0.6, use_colnames=True)
#frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
#frequent_itemsets1=frequent_itemsets[frequent_itemsets['length'] == 1]

In [35]:
#frequent_itemsets[ (frequent_itemsets['length'] >1 ) & #esto sería para k=2
                   #(frequent_itemsets['support'] >= 0.8) ]

Vale, hay un apartado que pide representar gráficamente las reglas: 

In [36]:
plot_confidence_vs_support(rulesaltaconfianza)

NameError: ignored

##Evaluación

##Despliegue